In [7]:
import pandas as pd
import numpy as np

inputs = pd.read_csv("../data/raw/inputs_prices.csv", sep=";")
commodities = pd.read_csv("../data/raw/commodities_prices.csv", sep=";")
fx = pd.read_csv("../data/raw/exchange_rate.csv", sep=";")

inputs["date"] = pd.to_datetime(inputs["date"])
commodities["date"] = pd.to_datetime(commodities["date"])
fx["date"] = pd.to_datetime(fx["date"])


In [8]:
assert inputs["currency"].isin(["USD", "ARS"]).all()
assert commodities["currency"].isin(["ARS"]).all()

In [16]:
inputs = inputs.sort_values("date")
commodities = commodities.sort_values("date")
fx = fx.sort_values("date")

In [17]:
inputs_fx = pd.merge_asof(
    inputs,
    fx,
    on="date",
    direction="backward"
)

commodities_fx = pd.merge_asof(
    commodities,
    fx,
    on="date",
    direction="backward"
)


In [18]:
inputs_fx["usd_ars_sell"].isna().mean()
commodities_fx["usd_ars_sell"].isna().mean()


np.float64(0.0)

In [19]:
inputs_fx["price_usd"] = np.where(
    inputs_fx["currency"] == "USD",
    inputs_fx["price"],
    inputs_fx["price"] / inputs_fx["usd_ars_sell"]
)

In [20]:
commodities_fx["price_usd"] = (
    commodities_fx["price"] / commodities_fx["usd_ars_sell"]
)

In [21]:
inputs_fx = inputs_fx.sort_values(["id_articulo", "date"])

inputs_fx["price_usd"] = (
    inputs_fx
        .groupby("id_articulo")["price_usd"]
        .ffill()
)

In [22]:
inputs_fx["year_month"] = inputs_fx["date"].dt.to_period("M")
commodities_fx["year_month"] = commodities_fx["date"].dt.to_period("M")

In [23]:
commodities_monthly = (
    commodities_fx
        .groupby(["commodity_id", "year_month"])
        .agg(price_usd_mean=("price_usd", "mean"))
        .reset_index()
)


In [24]:
inputs_monthly = (
    inputs_fx
        .sort_values("date")
        .groupby(["id_articulo", "year_month"])
        .agg(price_usd_last=("price_usd", "last"))
        .reset_index()
)


In [36]:
(
    inputs_monthly
    .groupby("id_articulo")["year_month"]
    .count()
    .sort_values(ascending=False)
    .head()
)


id_articulo
4955     90
41726    76
41740    73
4887     67
4939     58
Name: year_month, dtype: int64